In [ ]:
#This script preprocess comment and submission combined graph.
#Output preprocessed graphs should include input feature, class label, train index, test index, adjacency matrix

In [1]:
#load 2019 roberta model
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import os
from scipy.special import softmax
# Preprocess text (username and link placeholders)

path_preprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
path_model="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"
path_upprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"


os.chdir(path_model)

def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
#continue previous block
#preprocess using roBERT, turn comment and submission combined data to feature, and roberta score
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import punctuation
from autocorrect import Speller
from autocorrect import spell
from nltk.stem.porter import PorterStemmer
import heapq
import pickle
#import tensorflow_text
#use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")
#stopword = stopwords.words(‘english’)
import warnings

path_preprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\temp\\"
path_model="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"
path_upprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"

os.chdir(path_model)

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

years=['2019','2020']
schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
#schools = ["columbia","notredame","uofm"]
label_group=["label","unlabel"]
schoolyear=['dartmouth',"notredame2019","uofm2019","columbia2019","dartmouth2019","UCSD2019","berkeley2019","Harvard2019","ucla2019",
            "notredame2020","uofm2020","columbia2020","dartmouth2020","UCSD2020","berkeley2020","Harvard2020","ucla2020"]

#att_sub=["id","selftext","title"]
#drop_sub=["time","author_fullname","author","subreddit","subreddit_subscribers","all_awardings","allow_live_comments","can_mod_post","contest_mode","domain","gildings","is_crosspostable","is_meta","is_original_content","is_reddit_media_domain","is_robot_indexable","is_self","is_video","locked","media_only","no_follow","num_comments","num_crossposts","over_18","parent_whitelist_status","pinned","pwls","score","send_replies","spoiler","stickied","thumbnail","total_awards_received","whitelist_status","wls"]
att_com=["time","id","body","Emotion","Topic"]
drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]

files=[]
nb_cm=[0,0,0,0,0,0,0,0]
y=''
for s in schoolyear:
        f=s+y+".csv"
        os.chdir(path_upprocessed)
        data=pd.read_csv(f,skip_blank_lines=True)
        data=data[att_com]
        data['emo_pred_pos']=0
        data['emo_pred_neu']=0
        data['emo_pred_neg']=0
        
        for d in range(len(data['id'])):
            text = preprocess(data["body"][d])
            encoded_input = tokenizer(text, return_tensors='pt',max_length=512,truncation=True)
            output = model(**encoded_input,output_hidden_states=True)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            data.loc[d,'emo_pred_pos'] = round(float(scores[2]),4)
            data.loc[d,'emo_pred_neu'] = round(float(scores[1]),4)
            data.loc[d,'emo_pred_neg'] = round(float(scores[0]),4)
            
            
            hidden_states=output[-1]#[1][0][0].shape
            emb_torch=hidden_states[-1][0][0]
            emb = emb_torch.cpu().detach().numpy()
            os.chdir(path_preprocessed)
            name="feature_"+s+y+'.csv'
            e=open(name, 'a',newline='')
            with e:
                writer = csv.writer(e, delimiter=',')
                writer.writerow(emb)
        name=s+y+"_score.csv"
        os.chdir(path_preprocessed)
        data.to_csv(name, sep=',',mode='a',header=True)


In [ ]:
#form label for merged data
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import math
#stopword = stopwords.words(‘english’)
import warnings
import random
import scipy.sparse

#path_result='C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\results\\'
#path_data="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
#path_code="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"

path_preprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
path_model="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"
path_upprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
os.chdir(path_model)

years=['2019','2020']
schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
#schools = ["dartmouth"]

drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["time","id","body","Emotion","Topic"]

emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]

for s in schools:
    label_emt=[]
    idx=0
    for y in years:
        f=s+y+".csv"
        os.chdir(path_upprocessed)
        data=pd.read_csv(f,skip_blank_lines=True)
        data=data[att_com]
        for d in range(len(data['id'])):
            if isinstance(data['Emotion'][d],str)==False:
                row=[0,0]
            elif data['Emotion'][d]==emo[0]:
                row=[0,1]
            elif data['Emotion'][d]==emo[1]:
                row=[0,1]
            elif data['Emotion'][d]==emo[2]:
                row=[0,1]
            elif data['Emotion'][d]==emo[3]:
                row=[1,0]
            elif data['Emotion'][d]==emo[4]:
                row=[1,0]
                        
            os.chdir(path_preprocessed)  
            name="label_emt3_"+s+y+".csv"
            e=open(name, 'a',newline='')
            with e:
                writer = csv.writer(e, delimiter=',')
                writer.writerow(row)

#dartmouth
f="dartmouth.csv"
os.chdir(path_upprocessed)
data=pd.read_csv(f,skip_blank_lines=True)
data=data[att_com]
for d in range(len(data['id'])):
    if isinstance(data['Emotion'][d],str)==False:
        row=[0,0]
    elif data['Emotion'][d]==emo[0]:
        row=[0,1]
    elif data['Emotion'][d]==emo[1]:
        row=[0,1]
    elif data['Emotion'][d]==emo[2]:
        row=[0,1]
    elif data['Emotion'][d]==emo[3]:
        row=[1,0]
    elif data['Emotion'][d]==emo[4]:
        row=[1,0]
                        
    os.chdir(path_preprocessed)  
    name="label_emt3_dartmouth"+".csv"
    e=open(name, 'a',newline='')
    with e:
        writer = csv.writer(e, delimiter=',')
        writer.writerow(row)

In [ ]:
#form SYMMETRIC and ASYMMETRIEC adjacency matrix
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import punctuation
from autocorrect import Speller
from autocorrect import spell
from nltk.stem.porter import PorterStemmer
import scipy.sparse
#stopword = stopwords.words(‘english’)
import warnings
from scipy.sparse import csc_matrix
from scipy import sparse
from csv import reader

path_preprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
path_model="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\"
path_upprocessed="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
os.chdir(path_model)

types=["comment"]
#years=['2020']#['2019','2020']
years=['2019','2020']
schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
#schools = ["notredame"]

drop_com=["time","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["author_fullname","body","Emotion","Topic","id","link_id","parent_id"]

emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]
    
for s in schools:
    for y in years:
        dic_c={}
        idx=0
        f=s+y+".csv"
        os.chdir(path_upprocessed)
        data=pd.read_csv(f,skip_blank_lines=True)
        n=len(data['id'])
        for d in range(len(data['id'])):
            if isinstance(data['link_id'][d],str)==False:
                id_c="\""+"t3_"+data['id'][d][1:]
            else:
                id_c="\""+"t1_"+data['id'][d][1:]
            dic_c[id_c]=idx
            idx+=1
                         
        adj_cc=csc_matrix((n,n), dtype=np.int8)
        for d in range(len(data['id'])):
            if isinstance(data['link_id'][d],str)==False:
                id_c="\""+"t3_"+data['id'][d][1:]
            else:
                id_c="\""+"t1_"+data['id'][d][1:]
            parent=data['parent_id'][d]
            if parent in dic_c:
                adj_cc[dic_c[parent],dic_c[id_c]]=1
                adj_cc[dic_c[id_c],dic_c[parent]]=1
                
            parent=data['link_id'][d]
            if parent in dic_c:
                adj_cc[dic_c[parent],dic_c[id_c]]=1
                adj_cc[dic_c[id_c],dic_c[parent]]=1
        os.chdir(path_preprocessed)
        name="dic_Csym_"+s+y+".p"
        pickle.dump(dic_c,open(name,"wb"))
        name="CCsym_"+s+y+".npz"
        sparse.save_npz(name, adj_cc)
        
        
        adj_cc=csc_matrix((n,n), dtype=np.int8)
        for d in range(len(data['id'])):
            if isinstance(data['link_id'][d],str)==False:
                id_c="\""+"t3_"+data['id'][d][1:]
            else:
                id_c="\""+"t1_"+data['id'][d][1:]
            parent=data['parent_id'][d]
            if parent in dic_c:
                #adj_cc[dic_c[parent],dic_c[id_c]]=1
                adj_cc[dic_c[id_c],dic_c[parent]]=1
                
            parent=data['link_id'][d]
            if parent in dic_c:
                #adj_cc[dic_c[parent],dic_c[id_c]]=1
                adj_cc[dic_c[id_c],dic_c[parent]]=1
        os.chdir(path_preprocessed)
        name="dic_Casym_"+s+y+".p"
        pickle.dump(dic_c,open(name,"wb"))
        name="CCasym_"+s+y+".npz"
        sparse.save_npz(name, adj_cc)
        
        
        
#dartmouth
s='dartmouth'
y=''
dic_c={}
idx=0
f=s+y+".csv"
os.chdir(path_upprocessed)
data=pd.read_csv(f,skip_blank_lines=True)
n=len(data['id'])
for d in range(len(data['id'])):
    if isinstance(data['link_id'][d],str)==False:
        id_c="\""+"t3_"+data['id'][d][1:]
    else:
        id_c="\""+"t1_"+data['id'][d][1:]
    dic_c[id_c]=idx
    idx+=1
                         
adj_cc=csc_matrix((n,n), dtype=np.int8)
for d in range(len(data['id'])):
    if isinstance(data['link_id'][d],str)==False:
        id_c="\""+"t3_"+data['id'][d][1:]
    else:
        id_c="\""+"t1_"+data['id'][d][1:]
    parent=data['parent_id'][d]
    if parent in dic_c:
        adj_cc[dic_c[parent],dic_c[id_c]]=1
        adj_cc[dic_c[id_c],dic_c[parent]]=1
                
    parent=data['link_id'][d]
    if parent in dic_c:
        adj_cc[dic_c[parent],dic_c[id_c]]=1
        adj_cc[dic_c[id_c],dic_c[parent]]=1
os.chdir(path_preprocessed)
name="dic_Csym_"+s+y+".p"
pickle.dump(dic_c,open(name,"wb"))
name="CCsym_"+s+y+".npz"
sparse.save_npz(name, adj_cc)
        
        
adj_cc=csc_matrix((n,n), dtype=np.int8)
for d in range(len(data['id'])):
    if isinstance(data['link_id'][d],str)==False:
        id_c="\""+"t3_"+data['id'][d][1:]
    else:
        id_c="\""+"t1_"+data['id'][d][1:]
    parent=data['parent_id'][d]
    if parent in dic_c:
        #adj_cc[dic_c[parent],dic_c[id_c]]=1
        adj_cc[dic_c[id_c],dic_c[parent]]=1
                
    parent=data['link_id'][d]
    if parent in dic_c:
        #adj_cc[dic_c[parent],dic_c[id_c]]=1
        adj_cc[dic_c[id_c],dic_c[parent]]=1
os.chdir(path_preprocessed)
name="dic_Casym_"+s+y+".p"
pickle.dump(dic_c,open(name,"wb"))
name="CCasym_"+s+y+".npz"
sparse.save_npz(name, adj_cc)

In [ ]:
#assigning index
#index generating for neg VS non-neg
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import punctuation
from autocorrect import Speller
from autocorrect import spell
from nltk.stem.porter import PorterStemmer
import math
#stopword = stopwords.words(‘english’)
import warnings
import random
import scipy.sparse

#path_result='C:\\Backup of covid project\\2cls_posVSnonpos\\results\\'
#path_data="C:\\Backup of covid project\\2cls_posVSnonpos\\data\\"
#path_code="C:\\Backup of covid project\\"

path_result="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"
path_data="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data"
path_code="C:\\Backup of covid project\\2cls_CScombined_negVSnonneg\\data\\"

#path_result='C:\\Backup of covid project\\2cls_wlNeu_woNeu\\results_WOneu\\'
#path_data="C:\\Backup of covid project\\2cls_wlNeu_woNeu\\data\\"
#path_code="C:\\Backup of covid project\\"

random.seed(10)

types=["comment"]
years=['2019','2020']

label_group=["label","unlabel"]
months=['08','09','10','11']

drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["time","id","body","Emotion","Topic"]

#emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
emo=["Very negative","Negative","Neutral","Positive","Very Positive"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]




#FIRST DEAL WITH DARTMOUTH DATA
#schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
schools = ["dartmouth"]
for s in schools:
    train_index=[]
    test_index=[]
    to_be_labeled_index=[]
    idx=0
    n_test=[50,31,19]#this number is derived from proportion of neg, neu, and nonneg
    f=s+".csv"
    os.chdir(path_data)
    data=pd.read_csv(f,skip_blank_lines=True)
    for d in range(len(data['id'])):
        if isinstance(data['Emotion'][d],str)==False:
            to_be_labeled_index.append(idx)
        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
            test_index.append(idx)
            n_test[0]=n_test[0]-1
        elif data['Emotion'][d]==emo[0]:
            train_index.append(idx)
        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
            test_index.append(idx)
            n_test[0]=n_test[0]-1
        elif data['Emotion'][d]==emo[1]:
            train_index.append(idx)
        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
            test_index.append(idx)
            n_test[1]=n_test[1]-1
        elif data['Emotion'][d]==emo[2]:
            train_index.append(idx)
        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
            test_index.append(idx)
            n_test[2]=n_test[2]-1
        elif data['Emotion'][d]==emo[3]:
            train_index.append(idx)
        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
            test_index.append(idx)
            n_test[2]=n_test[2]-1
        elif data['Emotion'][d]==emo[4]:
            train_index.append(idx)
        else:
            to_be_labeled_index.append(idx)
        idx+=1
            
                           
    os.chdir(path_result)
    name="train_index_"+s+"_cm.p"
    pickle.dump(train_index,open(name,"wb"))
    name="test_index_"+s+"_cm.p"
    pickle.dump(test_index,open(name,"wb"))
    name="to_be_labeled_index_"+s+"_cm.p"
    pickle.dump(to_be_labeled_index,open(name,"wb"))
        
        
        
        
#FIRST DEAL WITH DARTMOUTH DATA
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla",'dartmouth']
#schools = ["dartmouth"]
for s in schools:
    for y in years:
        train_index=[]
        test_index=[]
        to_be_labeled_index=[]
        idx=0
        n_test=[50,31,19]#this number is derived from proportion of neg, neu, and nonneg
        f=s+y+".csv"
        os.chdir(path_data)
        data=pd.read_csv(f,skip_blank_lines=True)
        for d in range(len(data['id'])):
            if isinstance(data['Emotion'][d],str)==False:
                to_be_labeled_index.append(idx)
            elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                test_index.append(idx)
                n_test[0]=n_test[0]-1
            elif data['Emotion'][d]==emo[0]:
                train_index.append(idx)
            elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                test_index.append(idx)
                n_test[0]=n_test[0]-1
            elif data['Emotion'][d]==emo[1]:
                train_index.append(idx)
            elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                test_index.append(idx)
                n_test[1]=n_test[1]-1
            elif data['Emotion'][d]==emo[2]:
                to_be_labeled_index.append(idx)
            elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                test_index.append(idx)
                n_test[2]=n_test[2]-1
            elif data['Emotion'][d]==emo[3]:
                train_index.append(idx)
            elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                test_index.append(idx)
                n_test[2]=n_test[2]-1
            elif data['Emotion'][d]==emo[4]:
                train_index.append(idx)
            else:
                to_be_labeled_index.append(idx)
            idx+=1
        os.chdir(path_result)
        name="train_index_"+s+y+"_cm.p"
        pickle.dump(train_index,open(name,"wb"))
        name="test_index_"+s+y+"_cm.p"
        pickle.dump(test_index,open(name,"wb"))
        name="to_be_labeled_index_"+s+y+"_cm.p"
        pickle.dump(to_be_labeled_index,open(name,"wb"))